<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Personality%20Enabled%20MF/PEMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PEMF MODEL

In [1]:
# importing packages
import numpy as np
import numba as nb
import scipy.sparse as sp
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import scipy.spatial as spt
import statistics
import math
import json
from sklearn.model_selection import GridSearchCV
from random import sample

In [2]:
# importing packages
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.MENTION,p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from textblob import Word
import re
punctuation = re.compile(r'[-.?&!,:;()|0-9]')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
def pemf(R,ul,il,S,N,M,K,lambdaU,lambdaV,lambdaT,R_test,ult,ilt,St):
    def sigmoid(z):
        return 1 + ((5.0 - 1.0) / (1+np.exp(-z)))
    def dsigmoid(z):
        return (5.0 - 1.0) * np.exp(-z)/np.power((1+np.exp(-z)),2)
    def rmse(U,V,R):
        keylist = []
        dok_keys = np.array(R.todok().keys()).T.reshape([1,1])[0][0]
        for k in dok_keys:
          row = []
          for l in k:
            row.append(l)
          keylist.append(row) 
        keylist = np.array(keylist)
        utl = keylist[:, 0]
        itl = keylist[:, 1]
        error = (get_csrmat(sigmoid(U.dot(V.T)),utl,itl)-R).power(2).sum()/R.nnz
        error = np.sqrt((get_csrmat(sigmoid(U.dot(V.T)),utl,itl)-R).power(2)).sum()/np.sqrt(R.nnz)
        return np.sqrt(error)
    def mae(U,V,R):
        keylist = []
        dok_keys = np.array(R.todok().keys()).T.reshape([1,1])[0][0]
        for k in dok_keys:
          row = []
          for l in k:
            row.append(l)
          keylist.append(row) 
        keylist = np.array(keylist)
        utl = keylist[:, 0]
        itl = keylist[:, 1]
        error = abs(get_csrmat(sigmoid(U.dot(V.T)),utl,itl)-R).sum()/R.nnz 
        return error
    def get_csrmat(mat,ul,il):
        indx = ul*mat.shape[1]+il
        return sp.csr_matrix((np.take(np.array(mat),indx),(ul,il)),shape=(mat.shape[0],mat.shape[1]))
    def costL(U,V,S,ul,il):
        tmp = U.dot(V.T)
        Rx = get_csrmat(sigmoid(tmp),ul,il)
        cost = 0.5*((R - Rx).power(2)).sum()+0.5*lambdaU*np.linalg.norm(U)**2+0.5*lambdaV*np.linalg.norm(V)**2
        cost += 0.5*lambdaT*np.power(U-S.dot(U),2).sum()
        return cost
    def gradient(U,V,S,ul,il):
        dU = np.zeros(U.shape)
        dV = np.zeros(V.shape)
        dU = lambdaU*U
        tmp = U.dot(V.T)
        Rv = get_csrmat(dsigmoid(tmp),ul,il)
        Rx = get_csrmat(sigmoid(tmp),ul,il)
        dU += Rv.multiply((Rx-R)).dot(V)
        dU += lambdaT*(U-S.dot(U))-lambdaT*S.T.dot((U-S.dot(U)))
        dV = lambdaV*V
        dV += (Rv.multiply((Rx-R))).T.dot(U)
        if np.max(dU)>1:
            dU = dU/np.max(dU)
        if np.max(dV)>1:
            dV = dV/np.max(dV)
        return dU,dV


    def train(U,V,S,ul,il):
        res=[]
        steps=10
        rate = 1e-2
        pregradU = 0
        pregradV = 0
        tol=1e-6
        momentum = 0.9
        stage = max(steps/100 , 1)
        for step in range(steps):
            start = time.time()
            dU,dV = gradient(U,V,S,ul,il)
            dU = dU + momentum*pregradU
            dV = dV + momentum*pregradV
            pregradU = dU
            pregradV = dV
            if not step%stage and rate>0.001:
                rate = 0.95*rate
            U -= rate * dU
            V -= rate * dV
            e = costL(U,V,S,ul,il) / (len(U) * len(V))
            res.append(e)
            if not step%stage:
                #print("step",step,"error", e, "time", time.time() - start)
                #print("RMSE",rmse(U,V,R), "MAE",mae(U,V,R))
                e1 = e
            if step>150:# or abs(sum(res[-3:])-sum(res[-13:-10]))<tol:
                #print("====================")
                #print("stop in %d step"%(step))
                #print("error is ",e)
                #print("====================")
                break
        return U, V


    U = np.random.normal(0,0.01,size=(R.shape[0],K))
    V = np.random.normal(0,0.01,size=(R.shape[1],K))
    Ut = np.random.normal(0,0.01,size=(R_test.shape[0],K))
    Vt = np.random.normal(0,0.01,size=(R_test.shape[1],K))
    start = time.time()
    U,V = train(U,V,S,ul,il)

    #print("=================RESULT=======================")
    #print('K:%d,lambdaU:%s, lambdaV:%s,lambdaT:%s' \
    #        %(K,lambdaU,lambdaV,lambdaT))
    #print("mae",mae(U,V,R_test))
    #print("time",time.time() - start)
    #print("========================================")
    #print("mae recal", mean_absolute_error((U@V.T).toarray(), R_test))

    print("rmse ",rmse(Ut,Vt,R_test), "mae ",mae(Ut,Vt,R_test), "time",time.time() - start)
    return 0

In [4]:
def calc_distance(A,A1):
  similarity = np.dot(A, A1.T)
  square_mag = np.diag(similarity)
  inv_square_mag = 1 / square_mag
  inv_square_mag[np.isinf(inv_square_mag)] = 0
  inv_mag = np.sqrt(inv_square_mag)
  cosine = similarity * inv_mag
  cosine = cosine.T * inv_mag
  return cosine

# target

## data

In [5]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})
arr = tr[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating']]

In [6]:
# amazon review dataset magazines csv
path1 = r'/content/drive/MyDrive/Per_CD_RS/magazine_output_final.csv'
with open(path1, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile, nrows=100)
#arr = np.array(df[['0', '1', '2', '3', '4']])

#path2 = r'/content/drive/MyDrive/Per_CD_RS/Aaamzon_fashion_ru_tf.csv'
#with open(path2, encoding="utf-8", errors='ignore') as infile:
#  ru_tf_df = pd.read_csv(infile)
#ru_tf_df = ru_tf_df.drop(columns=['Unnamed: 0'])
#ru_tf = np.array(ru_tf_df)

In [7]:
# amazon review dataset movies json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_magazine_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile)
#az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [8]:
t = pd.merge(df, az, on='userId', how='inner')
arr = t[['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating']]
t.shape

(557, 12)

## preprocessing

In [ ]:
input_file =  r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Video_Games.json'
with open(input_file) as f:
    lines = f.read().splitlines()
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']
df_inter['json_element'].apply(json.loads)
az = pd.json_normalize(df_inter['json_element'].apply(json.loads))

arr1 = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})
x = pd.DataFrame(arr1.userId.value_counts()).reset_index()
y = x[x['userId']>=5]
arr2 = pd.DataFrame(y)
arr2 = arr2.rename(columns={"userId":"count","index":"userId"})
df = pd.merge(arr1, arr2, on='userId', how='inner')
df = df[df['rating']<6]
df = df[df['rating']>0]
x = pd.DataFrame(df.itemId.value_counts()).reset_index()
y = x[x['itemId']>=5]
arr3 = pd.DataFrame(y)
arr3 = arr3.rename(columns={"itemId":"count","index":"itemId"})
df2 = pd.merge(df, arr3, on='itemId', how='inner')
az = df2
tgt = az

tgt['processed_text'] = ""
tgt['reviewText'].fillna(" ")
array_text = []
for i in range(len(tgt)):
  x = tgt['reviewText'][i]
  word_tokens = word_tokenize(x) if type(x) != float else  " "
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  lemma_words = []
  for words in filtered_sentence:
      word = Word(words).lemmatize()
      lemma_words.append(word)
  punc_words = []
  for words in lemma_words:
      word =  punctuation.sub("", words)
      if len(word) > 0:
        punc_words.append(word.lower())
  line = ""
  for i in punc_words:
    line = line + " " + i.lower()
  array_text.append(line)

tgt['processed_text'] = array_text
az = tgt[['rating', 'userId', 'itemId', 'reviewText', 'processed_text']]
az = az.drop_duplicates(subset=None, keep='first')

In [ ]:
az = az[0:len(arr)]

In [ ]:
# amazon review dataset movies json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_fashion_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Movies_and_TV.json'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_json(infile, lines=True)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
src = az[['userId', 'itemId', 'rating']]
arr = pd.DataFrame(arr)
arr['userId'] = az['userId']
arr['itemId'] = np.array(az['itemId'])
arr['rating'] = az['rating']

## matrix factorization

In [9]:
# Construction of rating matrix wrt user u
ru = arr.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
ru_m = ru > 0
ru_m = ru_m.replace(True, 1)
ru_m = ru_m.replace(False, 0)
ru = np.array(ru)
ru_m = np.array(ru_m)

In [10]:
ru = arr.groupby(['userId', 'itemId'])['rating'].sum().unstack()
ru = pd.DataFrame(ru)
ru.shape

(94, 187)

In [11]:
# dividing into train and test

def split_data(ru, s, k):
  split = s
  k = k
  test = []
  set1  = ru.iloc[: , :int(ru.shape[1]/split)]
  set1 = set1.replace(np.nan, 0)
  for i in range(k):
    set2 = ru.iloc[: , i*int(ru.shape[1]/split):(i+1)*int(ru.shape[1]/split)]
    set2 = set2.replace(np.nan, 0)
    set1 = pd.concat([set1, set2], axis=1)
    set1  = ru.iloc[: , :int(ru.shape[1]/split)]
    set1 = set1.replace(np.nan, 0)
    set2  = ru.iloc[: , int(ru.shape[1]/split): 2*int(ru.shape[1]/split)]
    set2 = set2.replace(np.nan, 0)
    xx = pd.concat([set1, set2], axis=1)
    test.append(xx)
  return test

In [12]:
# with only trust and rating use this

def tgt_pred(l, k, a, lp, t, th, fold, split):

    train_size = split
    lr, k, alpha, lambda_p, lambda_t, thr = l, k, a, lp, t, th
    df_copy = arr.copy()
    train_set = df_copy.sample(frac=train_size).reset_index()
    user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
    #test_set = df_copy.drop(train_set.index).reset_index()
    test_set = pd.DataFrame(split_data(df_copy, (1-train_size)*10, fold)[0])
    user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))

    df = pd.DataFrame(user_features_train) 
    df.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
    df['userId'] = train_set.userId
    df2 = df.groupby(by='userId').mean().reset_index()
    df3 = np.array(df2[["open", 'cons', 'extra', 'agree', 'neuro']])
    

    
    ru = train_set.groupby(['userId', 'itemId'])['rating'].sum().unstack()
    ru = pd.DataFrame(ru)
    set1 = ru.iloc[: , :int(ru.shape[1]/(1-train_size)*10)]
    set1 = set1.replace(np.nan, 0)
    #for i in range(1,2):
    # set2 = ru.iloc[: , i*int(ru.shape[1]/10):(i+1)*int(ru.shape[1]/10)]
    #  set2 = set2.replace(np.nan, 0)
    #  set1 = pd.concat([set1, set2], axis=1)
    ru = set1
    ru_m = ru > 0
    ru_m = ru_m.replace(True, 1)
    ru_m = ru_m.replace(False, 0)
    ru = np.array(ru)
    ru_m = np.array(ru_m)

    R0 = ru    # rating matrix
    max_r = 5.0    # max _rating   
    N,M= ru.shape[0],ru.shape[1]     # # n : no of users   m : no of items 
    K = k                       #k : latent dimension        (mx10)@(10xn)
    lambdaU,lambdaV,lambdaT= lambda_p, lambda_p, lambda_t
    keys = []

    R=sp.dok_matrix((N,M))   # create sparse matrix for user x item
    T=sp.dok_matrix((N,N))   # create sparse matrix for trust among users user x user
    for i in range(len(R0)):    # no of users
        for j in range(len(R0[i])):   # no of items
            if R0[i][j]>0:    # if rating is present   
                keys.append([i,j])
                R[i,j] = R0[i][j]     

    dft = pd.DataFrame(user_features_test) 
    dft.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
    dft['userId'] = test_set.userId
    df2t = dft.groupby(by='userId').mean().reset_index()
    df3t = np.array(df2t[["open", 'cons', 'extra', 'agree', 'neuro']])

    rut = test_set.pivot_table(index='userId',columns='itemId',values='rating')
    rut = test_set.groupby(['userId', 'itemId'])['rating'].sum().unstack()
    rut = pd.DataFrame(rut)
    rut = rut.fillna(0)
    ru_mt = rut > 0
    ru_mt = ru_mt.replace(True, 1)
    ru_mt = ru_mt.replace(False, 0)
    rut = np.array(rut)
    ru_mt = np.array(ru_mt)

    R0t = rut    # rating matrix
    max_rt = 5.0    # max _rating   
    Nt,Mt= rut.shape[0],rut.shape[1]     # # n : no of users   m : no of items 
    K = k                        #k : latent dimension        (mxk)@(kxn)
    lambdaU,lambdaV,lambdaT = lambdaU,lambdaV,lambdaT
    keyst = []

    Rt = sp.dok_matrix((Nt,Mt))   # create sparse matrix for user x item
    Tt = sp.dok_matrix((Nt,Nt))   # create sparse matrix for user x user
    for i in range(len(R0t)):    # no of users
        for j in range(len(R0t[i])):   # no of items
            if R0t[i][j]>0:    # if rating is present   
                keyst.append([i,j])
                Rt[i,j] = R0t[i][j]      

    
    # get trust factor between users trust_uv(user x user)  and sim_uv(user x user) t_uv(user x user)


    trust_uv = 1 - calc_distance(df3, df3)
    trust_uv = np.nan_to_num(trust_uv)
    trust_uv_s = trust_uv.copy()
    trust_uv = pd.DataFrame(trust_uv >= thr)
    trust_uv = trust_uv.replace(True, 1)
    trust_uv = trust_uv.replace(False, 0)
    trust_uv = np.array(trust_uv)

    tknn = []
    for j in range(len(trust_uv)):
      row = []
      for m in range(len(trust_uv[j])):
        if trust_uv[j][m] == 1:
          row.append(m)
      tknn.append(row)
    tknn = np.array(tknn)
    sim_uv = 1- calc_distance(ru, ru)
    sim_uv = np.nan_to_num(sim_uv)
    #ru_tf2 = ru_tf[0:trust_uv.shape[0]]
    #ru_tf2 = ru_tf[:,:len(trust_uv)]
    t_uv = np.add(alpha * trust_uv_s, (1-alpha)* sim_uv)
    #t_uv = np.multiply(np.add(alpha * trust_uv_s, (1-alpha)* sim_uv), ru_tf)

    max_value = max(t_uv.flatten())
    min_value = min(t_uv.flatten())
    T_df = pd.DataFrame(t_uv)
    T_df = T_df >=  np.subtract(T_df, min_value) /(max_value - min_value)   # thr is threshold value
    T_df = T_df.replace(True, 1)
    T_df = T_df.replace(False, 0)
    T_df = np.array(T_df)
    T = sp.csr_matrix(T_df)   # fill up trust matrix from user realtionships   *******


    trust_uv_t = 1 - calc_distance(df3t, df3t)
    trust_uv_t = np.nan_to_num(trust_uv_t)
    trust_uv_st = trust_uv_t.copy()
    trust_uv_t = pd.DataFrame(trust_uv_t >= thr)
    trust_uv_t = trust_uv_t.replace(True, 1)
    trust_uv_t = trust_uv_t.replace(False, 0)
    trust_uv_t = np.array(trust_uv_t)

    tknn_t = []
    for j in range(len(trust_uv_t)):
      row = []
      for m in range(len(trust_uv_t[j])):
        if trust_uv_t[j][m] == 1:
          row.append(m)
      tknn_t.append(row)
    tknn_t = np.array(tknn)
    sim_uv_t = 1- calc_distance(rut, rut)
    sim_uv_t = np.nan_to_num(sim_uv_t)
    #ru_tf2 = ru_tf[0:trust_uv.shape[0]]
    #ru_tf2 = ru_tf[:,:len(trust_uv)]
    t_uv_t = np.add(alpha * trust_uv_st, (1-alpha)* sim_uv_t)
    #t_uv = np.multiply(np.add(alpha * trust_uv_s, (1-alpha)* sim_uv), ru_tf)

    max_value = max(t_uv_t.flatten())
    min_value = min(t_uv_t.flatten())
    T_df_t = pd.DataFrame(t_uv_t)
    T_df_t = T_df_t >=  np.subtract(T_df_t, min_value) /(max_value - min_value)   # thr is threshold value
    T_df_t = T_df_t.replace(True, 1)
    T_df_t = T_df_t.replace(False, 0)
    T_df_t = np.array(T_df_t)
    Tt = sp.csr_matrix(T_df_t)   # fill up trust matrix from user realtionships   *******


    
    keys = np.array(keys)        # get the keys
    keyst = np.array(keyst)
    R,T = R.tocsr(),T.tocsr()
    Rt,Tt = Rt.tocsr(), Tt.tocsr()
    pemf(R,keys[:, 0],keys[:, 1],T,N,M,K,lambdaU,lambdaV,lambdaT,Rt,keyst[:, 0],keyst[:, 1],Tt)

## Run

In [13]:
lr = [0.0001]
K = [40]
alpha = [0.5]
lambda_p = [0.02]
lambda_t = [0.5]
thres = [0.7]
fold = 1
split = 0.8

In [14]:
l, k, a, lp, lt, th = 0.0001, 20, 0.5, 0.03, 0.8, 0.8

In [15]:
if __name__ == "__main__":
  for i in range(fold):
    for l in lr:
      for k in K:
        for a in alpha:
          for lp in lambda_p:
            for lt in lambda_t:
              for th in thres:
                print(l, k, a, lp, lt, th)
                model = tgt_pred(l, k, a, lp, lt, th, fold, split)

0.0001 40 0.5 0.02 0.5 0.7
rmse  7.51462790680989 mae  2.8779589644793466 time 0.06507492065429688


In [ ]:
# @params: 
# trust threshold (0.7) t  ***
# mixing factor alpha (0.5)   ***
# lambdaU,lambdaV,lambdaT (0.02, 0.02, 0.1)
# no of iterations (100) +++
# no of latent features K (10) ***
# train test split  
# learning rate

# metrics
# MAE
# RMSE

# benchmark
# yakchi pacis
# tobias umap
# p2mf cdrup

# variations
# numerical personality values (0,1)
# binary personality values [0 and 1]
# with demography dbscan clustering
  # ensemble through average voting
  # ensemble through plurality voting 
# without clustering